## import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize
import scipy.stats
import math
import random
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn.model_selection import train_test_split
from itertools import combinations

from mpmath import mp
mp.dps = 50

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## class Model

In [2]:
class Model:
  
  def __init__(self,x_i,y_i,allowed_polynomials):
    ''' class representing a simple polynomial regression model

    Parameters
    ----------
    x: 1d numpy.array
        feature vector
    y: 1d numpy.array
        response vector
    allowed_polynomials: 1d list
        allowed degrees for polynomials of the feature vector
    '''

    self.x = x_i
    self.y = y_i
    self.allowed_polynomials = allowed_polynomials
    self.n_param = len(allowed_polynomials)
    self.likl=0
    self.params = []

    
  def fit(self):
    ''' fits the polynomial regression using ordinary least squares (OLS) from statsmodels.regression.linear_model 
    '''

    model_features = np.column_stack([self.x**i for i in self.allowed_polynomials])
    model_regr = OLS(self.y, model_features).fit()
    self.likl = mp.exp(-model_regr.bic/2)
    self.params = model_regr.params
    
    return self
  
  def is_subModel(self,other):
    ''' check if a model is a subModel of another one
    Parameters
    ----------
    other: Model
        the other model that will be checked against
    Returns
    -------
    True or False
    '''

    if len(self.allowed_polynomials) >= len(other.allowed_polynomials):
      return False
    else:
      for i in range(0,len(self.allowed_polynomials)):
        if not self.allowed_polynomials[i] in other.allowed_polynomials:
          return False
      return True
  
  def predict(self,x_i):

    y_pred = np.zeros(len(x_i))
    for i in range(0,len(x_i)):
      for j in range(0, len(self.allowed_polynomials)):
        y_pred[i]+= self.params[j] * np.power(x_i[i],self.allowed_polynomials[j])
    
    return y_pred


## class BMA_poly

In [3]:
class BMA_poly:
  
  def __init__(self,x_i,y_i,max_degree,c_cutoff):
    '''class repressenting Byaesian model averaging for polynomial curve fitting using the method known as "Occam's window". This implementation of Occam's window 
    rejects models that are 1/c_cutoff less likely than the most likely model, and rejects new complex models that are less likely than one of their submodels. 
    The priors are assumed to be uniform for this problem.

    Parameters
    ----------
    x: 1d numpy.array
        features vector
    y: 1d numpy.array
        response vector
    max_degree: float
        maximum degree for the polynomial regression (i.e. for max_degree = 10 we can have a polynomial of up to x^10)
    c_cutoff: float
        some models will be discarded if they predict the data far less well (i.e. if max_lik/lik_A > c_cutoff then model A will be discarded)
    
    '''

    self.x = x_i
    self.y = y_i
    self.max_degree = max_degree
    self.c_cutoff = c_cutoff
    self.good_models=[]
    self.sum_likl = 0

  
  def occam_razor(self, curr_model):
    '''excludes complex models which receive less support from the data than their simpler counterparts.
    Given the list of good_models is always ordered by the number of parameters all subModels will already be included and checked
    Parameters
    ----------
    model: Model
        model that needs to be checked

    Returns
    -------
    False if the model is too complex and True if the model is good and can be included
    '''
    for a_model in self.good_models:
      if a_model.is_subModel(curr_model) and a_model.likl > curr_model.likl:
        return False

    return True 

  def fit(self):
    '''perform the Bayesian model averaging 

    Returns
    -------
    self: BMA_poly
        Returns the new BMA_poly object as an output
    '''
     
    max_likl = 0
    
    # iterate through all possible models by going through the number of parameters (i.e. polynomial degrees)
    for num_parameters in range(1,self.max_degree+1): 

      # list of all models with fixed num_parameters (it will be a list of numbers that represent the allowed polynomial degrees from 0->20)
      current_list = list(combinations(list(range(self.max_degree)), num_parameters))

      for allowed_polynomials in current_list:
        curr_model = Model(self.x,self.y,allowed_polynomials).fit()
        
        # decide whether to save this model or not by first comparing its likl with the max_likl and then checking if there are any better subModels
        if max_likl/curr_model.likl < self.c_cutoff and self.occam_razor(curr_model):
          self.good_models.append(curr_model)

          # update the max_likl if needed and update the list of good models
          if curr_model.likl > max_likl:
            max_likl = curr_model.likl
            for a_model in self.good_models:
              if max_likl/a_model.likl > self.c_cutoff:
                self.good_models.remove(a_model)
 
    self.sum_likl = 0
    for a_model in self.good_models:
      self.sum_likl += a_model.likl
      
    # Return the new BMA object as an output.
    return self
  
  def predict(self, x_i):

    y_pred = np.zeros(len(x_i))
    for a_model in self.good_models:
      prob = a_model.likl/self.sum_likl
      for i in range(0,len(x_i)):
        y_pred[i] +=  prob * a_model.predict(x_i)[i]

    return y_pred


## tests

In [4]:
# generate random data
num_poly = 8 # number of polyomials
num_data = 200 # number of data points

weights = [np.random.uniform(-1,1) for i in range(0,num_poly)]
def f(x,weights):
    y = np.random.uniform(0,1)
    for i in range(0,len(weights)):
        y += weights[i]*np.power(x,i)
    return y

# generate some input data and add noise
input_data = [(lambda x:(x,f(x,weights)))(np.random.uniform(-5,5)) for i in range(0,num_data)]

x_data, y_data = [[ i for i, j in input_data ],[ j for i, j in input_data ]] #unzip the tuples in input_data
x_train = np.array(x_data)[:int(num_data/2)]
y_train = np.array(y_data)[:int(num_data/2)]
x_test = np.array(x_data)[int(num_data/2):]
y_test = np.array(y_data)[int(num_data/2):]

In [16]:
def train(x_ii, y_ii, x_to_test):
  result_BMA = BMA_poly(x_ii, y_ii, 10, 20).fit()
  result_MAP = result_BMA.good_models[0]
  max_likl = 0
  for a_model in result_BMA.good_models:
    if a_model.likl > max_likl:
      max_likl = a_model.likl
      result_MAP = a_model
  
  y_pred_BMA = result_BMA.predict(x_to_test)
  y_pred_MAP = result_MAP.predict(x_to_test)

  print("BMA gives models:")
  for i in range(0,len(result.good_models)):
    print(result.good_models[i].allowed_polynomials, "with probability of " + str(result.good_models[i].likl/result.sum_likl))

  print("\n")
  return y_pred_BMA, y_pred_MAP

In [17]:
# compare BMA_poly with the MAP method which only chooses the best among all models

y_1, y_2 = train(x_train, y_train, x_test)

print("BMA gives an rmse of:"+str(np.sqrt(sum((y_1 - y_test)**2)/len(y_test))))
print("MAP gives an rmse of:"+str(np.sqrt(sum((y_2 - y_test)**2)/len(y_test))))

BMA gives models:
(0, 1, 2, 3, 4, 5, 6, 7) with probability of 0.4030336661373285908062383521295004707988498575133
(0, 1, 2, 3, 4, 5, 6, 7, 8) with probability of 0.5969663338626714091937616478704995292011501424867


BMA gives an rmse of:0.3096121982859752
MAP gives an rmse of:0.3179593274264541
